In [ ]:
# 크롤링 ai사이트
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pyautogui

# 크롬 드라이버 초기화
driver = webdriver.Chrome()
cultures = ["문화회관", "도서관", "박물관", "기타", "공원장", "미술관", "문화원"]


addresses = ['중랑구','강남','강동','강북','강서','관악','광진','구로','금천','노원','도봉',
             '동대문','동작','마포','서대문','서초','성동','송파','양천','영등포',
             '용산','은평','종로']

# gangnam 카테고리 초기화
# gangnam = {"문화회관": 0, "도서관": 0, "박물관": 0, "기타": 0, "공원장": 0,"미술관": 0, "문화원": 0}
cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}
# for i in addresses:
    
# 데이터 읽기
data = pd.read_csv('filtered_data_중랑구.csv', encoding='utf-8')
# data = pd.read_excel('filtered_data_중랑구.xlsx')
urls = data['홈페이지']
position = data['문화시설명']

# CSV 파일에 헤더(컬럼 이름) 기록
with open(f'count_data_중랑구.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # 헤더 작성 (문화시설명 + gangnam의 키 값들)
    header = ['문화시설명'] + list(cultures_dic.keys())
    writer.writerow(header)

# 각 홈페이지를 순회하며 텍스트 분석
for homepage, facility_name in zip(urls, position):
    cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}
    print(position, end= ' ')
    try: 
        # 셀레니움으로 페이지 요청 및 로드
        driver.get(homepage)
        WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
        try:
            close_button = driver.find_element(By.XPATH, '//*[text()="닫기"]')
            for i in len(close_button):
                time.sleep(0.2)
                close_button.click()
        except:
            print('닫기 버튼 없음')

        # 페이지의 body 태그에서 모든 텍스트 가져오기
        body_text = driver.find_element(By.TAG_NAME, 'body').text
        
        body_list_1 = body_text.split('\t')
        body_list_2 = [item for sublist in body_list_1 for item in sublist.split(' ')]
        body_list_3 = [item for sublist in body_list_2 for item in sublist.split('\n')]
        body_set = set(body_list_3)
        print(len(body_set))
        print(position, end= ' ')
    except Exception as e:
        print(e)
    try:
        # body_list에서 각 줄을 확인하며 카테고리 찾기
        for body in body_set:
            driver.get('https://aiopen.etri.re.kr/demo/WordRel')
            
            try:
                word1 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[1]/td/input')
            except:
                word1 = driver.find_element(By.ID,'box3')
            # time.sleep(1)
            try:
                word2 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[2]/td/input')
            except:
                word2 = driver.find_element(By.ID,'word2')
            # time.sleep(1)

            try:
                button = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]/button')
            except:
                try:
                    button = driver.find_element(By.CLASS_NAME,'btnBlue')
                except:
                    button = driver.find_element(By.CSS_SELECTOR,'btnBlue')

            WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
            # time.sleep(1)
            word2.click()
            time.sleep(0.1)
            pyautogui.hotkey('ctrl', 'a')
            time.sleep(0.2)
            pyautogui.hotkey('del')
            word2.send_keys(body)
            # time.sleep(1)

            # 각 장소의 홈페이지에 어떤 키워드가 있는지 분석
            for category in cultures_dic.keys():
                word1.click()
                time.sleep(0.2)
                pyautogui.hotkey('ctrl', 'a')
                time.sleep(0.2)
                pyautogui.hotkey('del')
                word1.send_keys(category)
                #if category in body:
                # time.sleep(0.2)
                WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                button.click()
                time.sleep(0.1)

                try:
                    value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                except:
                    value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)    
                try:
                    value_2 = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[1]/tbody/tr/td[2]').text)
                except:
                    value_2 = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[1]/tbody/tr/td[2]').text)
                time.sleep(0.2)                                     
                try:
                    if value > 0.2:
                        cultures_dic[category] += 1
                        print(cultures_dic[category],end= ' ')
                except:
                    if value_2 > 0:
                        cultures_dic[category] += 1
                        print(cultures_dic[category], end=' ')
                time.sleep(0.1)
    except Exception as e:
        print(e)
                # 데이터 CSV 파일에 기록하기
    with open(f'count_data_중랑구.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # gangnam 값을 리스트로 변환하고 position과 결합
        row = [facility_name] + list(cultures_dic.values())
        writer.writerow(row)
    
# 결과 출력
# print)

# 크롬 드라이버 종료
driver.quit()


In [2]:
# 영상에 나오는 코드
!pip install requests beautifulsoup4 scikit-learn nltk

import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk

# NLTK의 punkt 데이터 다운로드
nltk.download('punkt')

# 웹 페이지의 URL
url = 'https://www.museum.go.kr/site/main/home'

# 웹 페이지 요청 및 파싱
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 페이지에서 텍스트 추출
text = soup.get_text()

# 텍스트를 단어 단위로 토큰화
words = word_tokenize(text)

# 관심 있는 카테고리 리스트
categories = [
    '전시회', '박물관', '음악', '무용', '전통예술', 
    '연극공연', '영화관람', '대중공연'
]

# 카테고리와 관련된 텍스트 비율 계산
def calculate_category_proportions(text, categories):
    category_counts = {category: 0 for category in categories}
    total_words = len(words)
    
    for category in categories:
        category_counts[category] = text.lower().count(category)
    
    proportions = {category: count / total_words for category, count in category_counts.items()}
    
    return proportions

# 카테고리 비율 계산
category_proportions = calculate_category_proportions(text, categories)

# 결과 출력
print("카테고리 비율:")
for category, proportion in category_proportions.items():
    print(f"{category}: {proportion:.4f}")

# TF-IDF 벡터화 및 분석
def get_tfidf_scores(text, categories):
    vectorizer = TfidfVectorizer(vocabulary=categories)
    X = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    
    scores = {category: X[0, i] for i, category in enumerate(feature_names)}
    
    return scores

# TF-IDF 점수 계산
category_scores = get_tfidf_scores(text, categories)

# 결과 출력
print("\n카테고리 TF-IDF 점수:")
for category, score in category_scores.items():
    print(f"{category}: {score:.4f}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vvv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\vvv/nltk_data'
    - 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\vvv\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# 문장 분석 모델
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

# spacy 모델 로드
nlp = spacy.load('ko_core_news_sm')

# 웹 페이지의 URL
url = 'https://www.museum.go.kr/site/main/home'

# 웹 페이지 요청 및 파싱
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 페이지에서 텍스트 추출
text = soup.get_text()

# 문장을 나누기 (spacy 사용)
def split_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

sentences = split_sentences(text)

# 관심 있는 카테고리 리스트
categories = [
    '전시회', '박물관', '음악', '무용', '전통예술', 
    '연극공연', '영화관람', '대중공연'
]

# TF-IDF 벡터화 및 분석
def get_tfidf_scores(sentences, categories):
    vectorizer = TfidfVectorizer(vocabulary=categories)
    X = vectorizer.fit_transform(sentences)
    feature_names = vectorizer.get_feature_names_out()
    
    scores = {category: [] for category in categories}
    
    for i, sentence in enumerate(sentences):
        tfidf_scores = X[i].toarray()[0]
        for j, category in enumerate(categories):
            score = tfidf_scores[j]
            if score > 0:
                scores[category].append((i, score, sentences[i]))
    
    return scores

# TF-IDF 점수 계산
category_scores = get_tfidf_scores(sentences, categories)

# 결과 출력
for category, score_data in category_scores.items():
    print(f"\n{category}:")
    if score_data:
        for index, score, sentence in sorted(score_data, key=lambda x: -x[1]):
            print(f"문장: {sentence}")
            print(f"점수: {score:.4f}")
    else:
        print("관련된 문장이 없습니다.")


In [ ]:
# 문장 분석 모델
import urllib3
import json
# API 요청 URL
openApiURL = "http://aiopen.etri.re.kr:8000/ParaphraseQA"

# ETRI에서 발급받은 Access Key
accessKey = "5d059aa9-b7b7-483a-ba61-e3572b9693e0"

# 분석할 문장들 예시 (전통과 한옥 관련 문장)
sentence1 = "과일"
sentence2 = '사과'

# API 요청 데이터 설정
requestJson = {
    "argument": {
        "sentence1": sentence1,
        "sentence2": sentence2
    }
}

# HTTP 통신 설정 및 요청
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8", "Authorization": accessKey},
    body=json.dumps(requestJson)
)

# 응답 결과 출력
print("[Response Code] " + str(response.status))
print("[Response Body]")
print(str(response.data, "utf-8"))
analize = set(str(response.data, "utf-8"))
